### Dependencies to Install / Initial Checks





In [1]:
!pip3 install datasets
!pip install numpy
!pip install sentence_transformers
!pip install --upgrade torch torchvision torchaudio
!pip install --upgrade transformers sentence-transformers
import torch
from datasets import Dataset
from sentence_transformers import (
    SentenceTransformer,
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments,
    SentenceTransformerModelCardData,
)
from sentence_transformers.losses import MultipleNegativesRankingLoss
from sentence_transformers.training_args import BatchSamplers, ParallelMode
from sentence_transformers.evaluation import TripletEvaluator
! pip install -U accelerate
! pip install -U transformers
import accelerate
import os

In [2]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free: 248.7 GB  | Proc size: 1.1 GB
GPU RAM Free: 14917MB | Used: 0MB | Util   0% | Total 15360MB


In [3]:
!nvidia-smi

Mon Jun 24 20:43:19 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   50C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
# Make Sure GPU Instance is Working
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    try:
        x = torch.tensor([1.0, 2.0, 3.0], device='cuda')
        print(f"Tensor on CUDA: {x}")
        torch.cuda.empty_cache()
        print("torch.cuda.empty_cache() executed successfully.")
    except Exception as e:
        print(f"Error during CUDA operations: {e}")

CUDA available: True
Tensor on CUDA: tensor([1., 2., 3.], device='cuda:0')
torch.cuda.empty_cache() executed successfully.


### Setting Up Dataset

In [5]:
#if missing place this txt file from the gdrive folder into the main area or change path if storing at different location.
file = open("/content/wiki_mixed_v1.txt", "r")

#dataset dictionary
dataset = {}
dataset["anchors"] = []
dataset["positive"] = []

#read dataset into dictionary
for line in file.readlines():
    if line == "Query|Document":
      continue
    else:
      if "|" in line:
        try:
          vals = line.split("|")
          query = vals[0]
          doc = vals[1]
          dataset["anchors"].append("search query: " + query)
          dataset["positive"].append("search document: " + doc)
        except:
          print("Error for line: ", line, line.count("|"))
dataset = Dataset.from_dict({
    "anchor": dataset["anchors"],
    "positive": dataset["positive"],
})

### Training

In [6]:
#loading nomic-ai/nomic-embed-text-v1 embeddings model for finetuning
!pip install einops
model = SentenceTransformer(
    "nomic-ai/nomic-embed-text-v1",
    trust_remote_code=True
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [7]:
loss = MultipleNegativesRankingLoss(model)

In [8]:
import os
os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '12355'
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"
# torch.distributed.init_process_group(rank = torch.cuda.device_count(), world_size=1)
print(accelerate.__version__)
args = SentenceTransformerTrainingArguments(
    # Required parameter:
    output_dir="models/nomic_dateaware",
    # Optional training parameters:
    num_train_epochs=1,
    per_device_train_batch_size=256, #in line with nomic embed technical report
    per_device_eval_batch_size=256,
    learning_rate=2e-5,
    warmup_ratio=0.1,
    weight_decay=0.01,
    warmup_steps=400,
    logging_first_step=True,
    fp16=False,  # Set to False if you get an error that your GPU can't run on FP16
    bf16=True,  # Set to True if you have a GPU that supports BF16
    batch_sampler=BatchSamplers.NO_DUPLICATES,  # MultipleNegativesRankingLoss benefits from no duplicate samples in a batch
    # Optional tracking/debugging parameters:
    eval_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=1,
    save_total_limit=30,
    logging_steps=1,
    run_name="nomic_dateaware",  # Will be used in W&B if `wandb` is installed
)
print(args.parallel_mode)
print("Initializing trainer...")

0.31.0
ParallelMode.NOT_DISTRIBUTED
Initializing trainer...


In [ ]:
import transformers
transformers.logging.set_verbosity_debug()
transformers.utils.logging.enable_default_handler()
def main():
  # Set environment variables for debugging
  os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
  os.environ['TORCH_USE_CUDA_DSA'] = "1"

  # Print CUDA and PyTorch version info
  # print(f"PyTorch version: {torch.__version__}")
  # print(f"CUDA version: {torch.version.cuda}")
  # print(f"CUDA available: {torch.cuda.is_available()}")

  # # Check GPU memory status
  # print(torch.cuda.memory_summary())

  # # Print model, args, dataset, and loss for inspection
  # print("Model:", model)
  # print("Args:", args)
  # print("Dataset:", dataset)
  # print("Loss:", loss)

  # Proceed with trainer initialization
  try:
        #Restart Session using the commented lines if this cell fails.
      # import os
      # os._exit(00)
      trainer = SentenceTransformerTrainer(
          model=model,
          args=args,
          train_dataset=dataset,
          loss=loss,
      )
      trainer.add_callback(transformers.ProgressCallback)
      print("Trainer initialized successfully.")
      # Proceed with training
      trainer.train()
      model.save_pretrained("models/nomic_dateaware/final")
      print("Model saved successfully.")
  except RuntimeError as e:
      print("Caught RuntimeError:", e)
      print("Potential issue with CUDA kernel execution.")
if __name__ == "__main__":
    main()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using auto half precision backend
Currently training with a batch size of: 1024
***** Running training *****
  Num examples = 127,682
  Num Epochs = 1
  Instantaneous batch size per device = 256
  Training with DataParallel so batch size has been adjusted to: 1,024
  Total train batch size (w. parallel, distributed & accumulation) = 1,024
  Gradient Accumulation steps = 1
  Total optimization steps = 125
  Number of trainable parameters = 136,731,648


Trainer initialized successfully.


  0%|          | 0/125 [00:00<?, ?it/s]